In [2]:
import pandas as pd

In [3]:
# Ground Truth CSV
PATH = "../Data/"
gt_csv = PATH + "Dataset/CSVs/test_with_bg.csv"

# Read the Ground Truth CSV
gt_df = pd.read_csv(gt_csv)

In [4]:
def group_consecutive_predictions(predictions_df):
    grouped_predictions = []
    current_group = []
    last_end = None
    for _, row in predictions_df.iterrows():
        if current_group and (row['Start (s)'] != last_end or row['Scientific name'] != current_group[-1]['Scientific name']):
            # New group starts here
            grouped_predictions.append(current_group)
            current_group = []
        current_group.append(row)
        last_end = row['End (s)']
    if current_group:  # Add last group
        grouped_predictions.append(current_group)
    # Combine groups in unique predictions
    combined_predictions = []
    for group in grouped_predictions:
        combined_prediction = {
            'Start (s)': group[0]['Start (s)'],
            'End (s)': group[-1]['End (s)'],
            'Scientific name': group[0]['Scientific name'],
            'Confidence': max(item['Confidence'] for item in group)  # conf = max confidence in group
        }
        combined_predictions.append(combined_prediction)
    return combined_predictions

# Function to calculate the IoU
def calculate_iou(interval1, interval2):
    start_max = max(interval1[0], interval2[0])
    end_min = min(interval1[1], interval2[1])
    intersection = max(0, end_min - start_max)
    union = (interval1[1] - interval1[0]) + (interval2[1] - interval2[0]) - intersection
    return intersection / union if union != 0 else 0

# Function to check if a prediction is correct
def is_prediction_correct_detector(prediction, gt_annotation, iou_threshold):
    iou = calculate_iou((prediction['Start (s)'], prediction['End (s)']), (gt_annotation['start_time'], gt_annotation['end_time']))
    return iou >= iou_threshold

# Function to check if a prediction is correct
def is_prediction_correct(prediction, gt_annotation, iou_threshold):
    iou = calculate_iou((prediction['Start (s)'], prediction['End (s)']), (gt_annotation['start_time'], gt_annotation['end_time']))
    prediction_class = prediction['Scientific name'].lower()
    gt_class = gt_annotation['specie'].lower()
    scientific_name_matches = prediction_class == gt_class
    return iou >= iou_threshold and scientific_name_matches

In [5]:
# Variables for metrics
correct_predictions = 0
total_predictions = 0
total_predictions_score = 0
correct_predictions_detector = 0
iou_threshold = 0.4  # This value is editable
prediction_conf_score = 0.4  # This value is editable

# Process the predictions
for _, gt_annotation in gt_df.iterrows():
    prediction_path = PATH + f"Dataset/BirdNET_Predictions/{gt_annotation['path'].replace('.WAV', '.BirdNET.results.csv')}"
    try:
        predictions_df = pd.read_csv(prediction_path)
        grouped_predictions = group_consecutive_predictions(predictions_df)  # Group predictions
        predictions_df = pd.DataFrame(grouped_predictions)  # Convert list dict to DataFrame

        for _, prediction in predictions_df.iterrows():
            total_predictions += 1
            if prediction['Confidence'] >= prediction_conf_score:
                total_predictions_score += 1
                if is_prediction_correct(prediction, gt_annotation, iou_threshold):
                    correct_predictions += 1
                if is_prediction_correct_detector(prediction, gt_annotation, iou_threshold):
                    correct_predictions_detector += 1
    except FileNotFoundError:
        print(f"Prediction file not found: {prediction_path}")

# Calculate and display the metrics
print("================== Metrics ==================\n")
print(f"Total Predictions: {total_predictions}")
print(f"Total Predictions with Score >= {prediction_conf_score}: {total_predictions_score}")
print(f"Total GT: {len(gt_df)}")
print(f"Correct Predictions Detector: {correct_predictions_detector}")
print(f"Correct Predictions Detector + Classifier: {correct_predictions}")

# Additional calculations for precision, recall, and F1-score
print("\n================== Detector Metrics ==================\n")
true_positives = correct_predictions_detector
false_positives = total_predictions_score - correct_predictions_detector
false_negatives = len(gt_df) - correct_predictions_detector
true_negatives = 0

# Calculate and display the metrics
accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives) if true_positives + true_negatives + false_positives + false_negatives != 0 else 0
precision = true_positives / (true_positives + false_positives) if true_positives + false_positives != 0 else 0
recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives != 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if precision + recall != 0 else 0
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1_score}")

print("\n================== Detector + Classifier Metrics ==================\n")
true_positives = correct_predictions
false_positives = total_predictions_score - correct_predictions
false_negatives = len(gt_df) - correct_predictions
true_negatives = 0

# Calculate and display the metrics
accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives) if true_positives + true_negatives + false_positives + false_negatives != 0 else 0
precision = true_positives / (true_positives + false_positives) if true_positives + false_positives != 0 else 0
recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives != 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if precision + recall != 0 else 0
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1_score}")

================== Metrics ==================

Total Predictions: 2303
Total Predictions with Score >= 0.4: 2303
Total GT: 542
Correct Predictions Detector: 40
Correct Predictions Detector + Classifier: 10

================== Detector Metrics ==================

Accuracy: 0.0142602495543672
Precision: 0.017368649587494574
Recall: 0.07380073800738007
F1-Score: 0.0281195079086116

================== Detector + Classifier Metrics ==================

Accuracy: 0.003527336860670194
Precision: 0.004342162396873643
Recall: 0.01845018450184502
F1-Score: 0.0070298769771529


In [ ]:
# Hacer con las predicciones de yolo como detectorr

# Correr inference.py desde la VPN, intentar encontrar mejores valores de iou y conf y poner esos para las métricas aquí, luego coger esos valores y ponerlos

# predictionms de val de yolov8:
'''
[{"image_id": "AM4_20230531_110000", "category_id": 0, "bbox": [301.605, 0.0, 31.041, 459.45], "score": 0.5276}, {"image_id": "AM15_20230712_074000", "category_id": 0, "bbox": [61.011, 0.439, 175.454, 460.228], "score": 0.48407}, {"image_id": "AM15_20230330_070000", "category_id": 0, "bbox": [57.914, 0.573, 30.844, 461.131], "score": 0.65271}, {"image_id": "AM15_20230330_070000", "category_id": 0, "bbox": [231.294, 1.093, 31.147, 460.902], "score": 0.53149}, {"image_id": "AM15_20230330_070000", "category_id": 0, "bbox": [620.696, 2.401, 34.881, 459.599], "score": 0.44922}, {"image_id": "AM8_20230304_093000", "category_id": 0, "bbox": [765.516, 0.0, 162.609, 460.58], "score": 0.63678}, 
'''

# Y si se hace predict sobre la carpeta test??


# Copilot:
'''
Ahoar quiero modificar estas funciones:

def group_consecutive_predictions(predictions_df):
    grouped_predictions = []
    current_group = []
    last_end = None
    for _, row in predictions_df.iterrows():
        if current_group and (row['Start (s)'] != last_end or row['Scientific name'] != current_group[-1]['Scientific name']):
            # New group starts here
            grouped_predictions.append(current_group)
            current_group = []
        current_group.append(row)
        last_end = row['End (s)']
    if current_group:  # Add last group
        grouped_predictions.append(current_group)
    # Combine groups in unique predictions
    combined_predictions = []
    for group in grouped_predictions:
        combined_prediction = {
            'Start (s)': group[0]['Start (s)'],
            'End (s)': group[-1]['End (s)'],
            'Scientific name': group[0]['Scientific name'],
            'Confidence': max(item['Confidence'] for item in group)  # conf = max confidence in group
        }
        combined_predictions.append(combined_prediction)
    return combined_predictions

# Function to calculate the IoU
def calculate_iou(interval1, interval2):
    start_max = max(interval1[0], interval2[0])
    end_min = min(interval1[1], interval2[1])
    intersection = max(0, end_min - start_max)
    union = (interval1[1] - interval1[0]) + (interval2[1] - interval2[0]) - intersection
    return intersection / union if union != 0 else 0

# Function to check if a prediction is correct
def is_prediction_correct_detector(prediction, gt_annotation, iou_threshold):
    iou = calculate_iou((prediction['Start (s)'], prediction['End (s)']), (gt_annotation['start_time'], gt_annotation['end_time']))
    return iou >= iou_threshold

# Function to check if a prediction is correct
def is_prediction_correct(prediction, gt_annotation, iou_threshold):
    iou = calculate_iou((prediction['Start (s)'], prediction['End (s)']), (gt_annotation['start_time'], gt_annotation['end_time']))
    prediction_class = prediction['Scientific name'].lower()
    gt_class = gt_annotation['specie'].lower()
    scientific_name_matches = prediction_class == gt_class
    return iou >= iou_threshold and scientific_name_matches

    Y este codigo:

    # Variables for metrics
correct_predictions = 0
total_predictions = 0
total_predictions_score = 0
correct_predictions_detector = 0
iou_threshold = 0.4  # This value is editable
prediction_conf_score = 0.4  # This value is editable

# Process the predictions
for _, gt_annotation in gt_df.iterrows():
    prediction_path = PATH + f"Dataset/BirdNET_Predictions/{gt_annotation['path'].replace('.WAV', '.BirdNET.results.csv')}"
    try:
        predictions_df = pd.read_csv(prediction_path)
        grouped_predictions = group_consecutive_predictions(predictions_df)  # Group predictions
        predictions_df = pd.DataFrame(grouped_predictions)  # Convert list dict to DataFrame

        for _, prediction in predictions_df.iterrows():
            total_predictions += 1
            if prediction['Confidence'] >= prediction_conf_score:
                total_predictions_score += 1
                if is_prediction_correct(prediction, gt_annotation, iou_threshold):
                    correct_predictions += 1
                if is_prediction_correct_detector(prediction, gt_annotation, iou_threshold):
                    correct_predictions_detector += 1
    except FileNotFoundError:
        print(f"Prediction file not found: {prediction_path}")

# Calculate and display the metrics
print("================== Metrics ==================\n")
print(f"Total Predictions: {total_predictions}")
print(f"Total Predictions with Score >= {prediction_conf_score}: {total_predictions_score}")
print(f"Total GT: {len(gt_df)}")
print(f"Correct Predictions Detector: {correct_predictions_detector}")
print(f"Correct Predictions Detector + Classifier: {correct_predictions}")

# Additional calculations for precision, recall, and F1-score
print("\n================== Detector Metrics ==================\n")
true_positives = correct_predictions_detector
false_positives = total_predictions_score - correct_predictions_detector
false_negatives = len(gt_df) - correct_predictions_detector
true_negatives = 0

# Calculate and display the metrics
accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives) if true_positives + true_negatives + false_positives + false_negatives != 0 else 0
precision = true_positives / (true_positives + false_positives) if true_positives + false_positives != 0 else 0
recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives != 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if precision + recall != 0 else 0
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1_score}")

print("\n================== Detector + Classifier Metrics ==================\n")
true_positives = correct_predictions
false_positives = total_predictions_score - correct_predictions
false_negatives = len(gt_df) - correct_predictions
true_negatives = 0

# Calculate and display the metrics
accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives) if true_positives + true_negatives + false_positives + false_negatives != 0 else 0
precision = true_positives / (true_positives + false_positives) if true_positives + false_positives != 0 else 0
recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives != 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if precision + recall != 0 else 0
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1_score}")

Para que funcione sobre el fichero predictions.json con la estructura de datos:
[{"image_id": "AM4_20230531_110000", "category_id": 0, "bbox": [301.605, 0.0, 31.041, 459.45], "score": 0.5276}, {"image_id": "AM15_20230712_074000", "category_id": 0, "bbox": [61.011, 0.439, 175.454, 460.228], "score": 0.48407}, {"image_id": "AM15_20230330_070000", "category_id": 0, "bbox": [57.914, 0.573, 30.844, 461.131], "score": 0.65271}, {"image_id": "AM15_20230330_070000", "category_id": 0, "bbox": [231.294, 1.093, 31.147, 460.902], "score": 0.53149}, {"image_id": "AM15_20230330_070000", "category_id": 0, "bbox": [620.696, 2.401, 34.881, 459.599], "score": 0.44922}, {"image_id": "AM8_20230304_093000", "category_id": 0, "bbox": [765.516, 0.0, 162.609, 460.58], "score": 0.63678}, ...

En la que 
'''

In [ ]:
# Hacer con las predicciones de BirdNet después de haber entrenado